# Model of Adaptive Resistance in Melanoma

In this notebook we will provide a step-by-step construction of the MARM model. The model is constructed using the rule-based modeling tool PySB <cite data-cite="2754712/L894YC64"></cite>. To start, we import all required pysb classes and instantiate the model:

In [1]:
from pysb import Model, Monomer, Parameter, Compartment, Expression,  Rule, Observable, Initial, Annotation, EnergyPattern, ANY
from sympy import exp, log

Model();

All model reactions are implemented in four different compartments, the extracellular compartment `EC`, the plasma membrane `PM`, the cytopplasm `CP` and the endosome membrane `EM`.

In [2]:
Compartment(name='EC', parent=None, dimension=3, size=None)
Compartment(name='PM', parent=EC, dimension=2, size=None)
Compartment(name='CP', parent=PM, dimension=3, size=None)
Compartment(name='EM', parent=CP, dimension=2, size=None)

Compartment(name='EM', parent=CP, dimension=2, size=None)

In the following, we split the model into individual submodules of strongly interacting components, which we will describe individually. The model describes protein and drug species as intracellular concentrations in uM at a volume of `1pL = 1e-12L`. For ligands and drugs we assume that the extracellular compartment is much bigger than the intracellular compartement and that equilibration is fast such that concentrations can be assumed to be constanct over time (implemented by specifying `fixed=True` in the initialization). For EGF, the parameter that specifies the abundance, `EGF_0` is assumed to have units [ng/ml] and is accordingly transformed into [uM] using the appropriate molecular weight `m_Da_EGF`. The Monomers (`RAFi`) and (`MEKi`) are placeholdes for different RAF inihibitors, such as vemurafenib, and MEK inhibitors, such as cobimetinib. the respective parameters, `RAFi_0` and `MEKi_0`, are assumed to be in [uM]. The unit of time is hours.

In [3]:
Expression('N_Avogadro', 6.02214076000000e+23)
Expression('volume', 1.00000000000000e-12)
Expression('m_Da_EGF', 6200.00000000000)

Monomer('RAFi', ['raf'])
Annotation(RAFi, 'http://identifiers.org/chebi/63637', 'is')
Parameter('RAFi_0')
Expression('initRAFi', RAFi_0)
Initial(RAFi(raf=None) ** CP, initRAFi, fixed=True)

Monomer('MEKi', ['mek'])
Annotation(MEKi, 'http://identifiers.org/chebi/90851', 'is')
Parameter('MEKi_0')
Expression('initMEKi', MEKi_0)
Initial(MEKi(mek=None) ** CP, initMEKi, fixed=True)

Monomer('EGF', ['rtk'])
Annotation(EGF, 'http://identifiers.org/uniprot/P01133', 'is')
Parameter('EGF_0')
Expression('initEGF', 6.02214076208112e+23*EGF_0/(N_Avogadro*m_Da_EGF))
Initial(EGF(rtk=None) ** CP, initEGF, fixed=True)

Initial(EGF(rtk=None) ** CP, initEGF, fixed=True)

# EGFR Signaling
The EGFR submodule describes the interaction between Egfr, Egf, Grb2, Cbl, Sos1 and Ras (which describes the action K-Ras, N-Ras and H-Ras). Here, we instantiate
the respective monomers and provide uniprot IDs as monomer annotations.

In [4]:
Monomer('EGFR', ['KD', 'Tyr', 'rtkf', 'ub'], {'Tyr': ['p', 'u'], 'ub': ['false', 'true']})
Annotation(EGFR, 'http://identifiers.org/uniprot/P00533', 'is')

Monomer('GRB2', ['SH2', 'SH3'])
Annotation(GRB2, 'http://identifiers.org/uniprot/P62993', 'is')

Monomer('SOS1', ['S1134', 'SH3m', 'ras'], {'S1134': ['p', 'u']})
Annotation(SOS1, 'http://identifiers.org/uniprot/Q07889', 'is')

Monomer('RAS', ['raf', 'sos1', 'state'], {'state': ['gdp', 'gtp']})
Annotation(RAS, 'http://identifiers.org/uniprot/P01111', 'is')
Annotation(RAS, 'http://identifiers.org/uniprot/P01116', 'is')
Annotation(RAS, 'http://identifiers.org/uniprot/P01112', 'is')

Monomer('CBL', ['SH3m'])
Annotation(CBL, 'http://identifiers.org/uniprot/P22681', 'is');

## EGF activation
To describe activation of Egfr through Egf, we implement base binding rates between Egf and Egfr. We implement these rules as energy rules, which allow us to later use the energy-based formalism in BioNetGen <cite data-cite="2754712/JHN992B6"></cite> to describe allosteric interactions between the two molecules. Energy rules
are parameterised using three distinct parameters: the `kD` parameter which describes the affinity between the two molecules, the `kf` parameter which describes the timescale of the interaction and the `phi` parameter which takes values between 0 and 1 and balance whether allosteric interactions modulate the affinity by changing the association rate (`kf`, only association if `phi=0`) or the dissociation rate (`kr = kf*kD`, only dissociation if `phi=1`).

We implement the ligand-receptor binding using the generic `rtk` binding site on the ligand and the `rtkf` binding site on the receptor. We implement the receptor-receptor binding using the `rtk` binding site on the receptor. This formalism simplifies future extensions of the model to additional ligands or receptors by ensuring that each receptor can only bind one ligand and one receptor. For receptor-receptor interactions, we implement asymmetric binding between kinase domain `KD` and the generic phosphorylation site `Tyr`. Implementing seperate reactions for `EGFR(KD=None, Tyr='u')` and `EGFR(KD=None, Tyr='p')` avoids infinite multimerization between EGFR molecules that would prevent network generation.

In [5]:
# binding Egf-Egfr
Parameter('bind_EGF_EGFR_kf')
Parameter('bind_EGF_EGFR_kD')
Parameter('bind_EGF_EGFR_phi')
Expression('bind_EGF_EGFR_kr', bind_EGF_EGFR_kf*bind_EGF_EGFR_kD)
Expression('Gf_bind_EGF_EGFR', log(bind_EGF_EGFR_kD))
Expression('Ea0_bind_EGF_EGFR', -(bind_EGF_EGFR_phi*log(bind_EGF_EGFR_kD) + log(bind_EGF_EGFR_kf)))
Rule('EGF_and_EGFR_bind_and_dissociate', 
     EGF(rtk=None) + EGFR(rtkf=None) | EGF(rtk=1) % EGFR(rtkf=1), 
     bind_EGF_EGFR_phi, Ea0_bind_EGF_EGFR, energy=True)
EnergyPattern('ep_bind_EGF_EGFR', EGF(rtk=1) % EGFR(rtkf=1), Gf_bind_EGF_EGFR)

# binding Egfr-Egfr
Parameter('bind_EGFR_EGFR_kf')
Parameter('bind_EGFR_EGFR_kD')
Parameter('bind_EGFR_EGFR_phi')
Expression('Gf_bind_EGFR_EGFR', log(bind_EGFR_EGFR_kD))
Expression('Ea0_bind_EGFR_EGFR', -(bind_EGFR_EGFR_phi*log(bind_EGFR_EGFR_kD) + log(bind_EGFR_EGFR_kf)))
Rule('bind_EGFR_EGFRu', EGFR(KD=None, Tyr='u') + EGFR(KD=None, Tyr='u') | EGFR(KD=1, Tyr='u') % EGFR(KD=None, Tyr=('u', 1)), bind_EGFR_EGFR_phi, Ea0_bind_EGFR_EGFR, energy=True)
Rule('bind_EGFR_EGFRp', EGFR(KD=None, Tyr='p') + EGFR(KD=None, Tyr='u') | EGFR(KD=1, Tyr='p') % EGFR(KD=None, Tyr=('u', 1)), bind_EGFR_EGFR_phi, Ea0_bind_EGFR_EGFR, energy=True)
EnergyPattern('ep_bind_EGFR_EGFR', EGFR(KD=1) % EGFR(Tyr=1), Gf_bind_EGFR_EGFR);

To implement allosteric interaction between Egf ligands and Egfr <cite data-cite="2754712/BIUJJCT7"></cite>, we add one energy patterns that allows the modulation of the affinities of Egfr-Egf and Egfr-Egfr interactions through binding of ligands.

In [6]:
# Egf mediated affinity modulation
Parameter('ep_EGFR_EGFR_mod_EGF_single_deltaG')
Expression('ep_EGFR_EGFR_EGF_Gf', log(ep_EGFR_EGFR_mod_EGF_single_deltaG))
EnergyPattern('ep_EGFR_EGFR_EGF', EGF(rtk=1) % EGFR(KD=ANY, rtkf=1), ep_EGFR_EGFR_EGF_Gf)
Annotation(ep_EGFR_EGFR_EGF, 'http://identifiers.org/pubmed/16946702', 'isDescribedBy')
Annotation(ep_EGFR_EGFR_EGF, 'http://identifiers.org/pubmed/16946702', 'isDescribedBy');

## EGFR phosphorylation
Egfr molecules transactivate each other following binding of Egf ligands. To simplify the model here, we implement a generic Tyrosine phosphorylation site that accounts for multiple different sites including Y992, Y1068, Y1086, Y1148, and Y1173. Dephosphorylation is implement as first order kinetics. Note that only specifying the state, in contrast to `('p',WILD)`, means that dephosphorylation will not take place if there is a bond involving that site.

In [7]:
Parameter('catalyze_EGFR_EGFR_p_kcat')
Parameter('catalyze_EGFR_u_kcatr')

Expression('catalyze_EGFR_u_kcat', 
           catalyze_EGFR_EGFR_p_kcat*catalyze_EGFR_u_kcatr)

Rule('catalyze_EGFR_EGFR_p', 
     EGFR(KD=1, rtkf=ANY) % EGFR(Tyr=('u', 1)) >> EGFR(KD=None, rtkf=ANY) + EGFR(Tyr='p'), 
     catalyze_EGFR_EGFR_p_kcat)
Rule('catalyze_EGFR_u', 
     EGFR(Tyr='p') >> EGFR(Tyr='u'), 
     catalyze_EGFR_u_kcat);

## GRB2 recruitement
Recruitement of Grb2 to receptor requires transactivation <cite data-cite="2754712/XCZ7WMHE"></cite> of multiple Egfr phosphorylation sites that are recognized by the Grb2 Src2 homology 2 (SH2) domain <cite data-cite="2754712/XPQTB6V8"></cite>. As the Grb2 SH2 uses the `Tyr` site as epitope for binding, this implicitely prevents dephosphorylation, and we can use the energy formalism to describe the interaction since both binding and unbinding have symmetric conditions.

In [8]:
# EGFR-Grb2 binding

Parameter('bind_pEGFR_GRB2_kf', 10.0)
Parameter('bind_pEGFR_GRB2_kD', 100.0)
Parameter('bind_pEGFR_GRB2_phi', 1.0)

Expression('Ea0_bind_pEGFR_GRB2', 
           -bind_pEGFR_GRB2_phi*log(bind_pEGFR_GRB2_kD) - log(bind_pEGFR_GRB2_kf))
Expression('Gf_bind_pEGFR_GRB2', 
           log(bind_pEGFR_GRB2_kD))

Rule('bind_pEGFR_GRB2', 
     GRB2(SH2=None) + EGFR(Tyr='p') | GRB2(SH2=1) % EGFR(Tyr=('p', 1)), 
     bind_pEGFR_GRB2_phi, Ea0_bind_pEGFR_GRB2, energy=True)
EnergyPattern('ep_bind_pEGFR_GRB2', EGFR(Tyr=1) % GRB2(SH2=1), Gf_bind_pEGFR_GRB2)

Annotation(bind_pEGFR_GRB2, 'http://identifiers.org/pubmed/16777603', 'isDescribedBy');

## CBL recruitement and EGFR ubiquitination
CBL is and E3-ligase that ubiquitinates EGFR, leading to degradation during endocytosis. CBL is recruited to EGFR by binding to the GRB2 SH3 domain. We implemented this interaction using an energy binding rule.

In [9]:
Parameter('bind_CBL_GRB2_kf')
Parameter('bind_CBL_GRB2_kD')
Parameter('bind_CBL_GRB2_phi')

Expression('bind_CBL_GRB2_kr', bind_CBL_GRB2_kD*bind_CBL_GRB2_kf)
Expression('Gf_bind_CBL_GRB2', log(bind_CBL_GRB2_kD))
Expression('Ea0_bind_CBL_GRB2', -bind_CBL_GRB2_phi*log(bind_CBL_GRB2_kD) - log(bind_CBL_GRB2_kf))

Rule('CBL_and_GRB2_bind_and_dissociate', CBL(SH3m=None) + GRB2(SH3=None) | CBL(SH3m=1) % GRB2(SH3=1), bind_CBL_GRB2_phi, Ea0_bind_CBL_GRB2, energy=True)
EnergyPattern('ep_bind_CBL_GRB2', CBL(SH3m=1) % GRB2(SH3=1), Gf_bind_CBL_GRB2);

To implement ubiquitination, we only require that CBL is in complex with EGFR. This means that in an EGFR dimer, CBL can ubiquitinate both EGFR molecules. EGFR deubiquitination is implemented as first order reaction. EGFR synthesis is desribed in a separate Section [Egfr expression](#Egfr expression)

In [10]:
Parameter('catalyze_CBL_EGFR_ub_kcat')
Parameter('catalyze_EGFR_dub_kcatr')

Expression('catalyze_EGFR_dub_kcat', catalyze_CBL_EGFR_ub_kcat*catalyze_EGFR_dub_kcatr)

Rule('catalyze_CBL_EGFR_ub', EGFR(ub='false') % CBL() >> EGFR(ub='true') % CBL(), catalyze_CBL_EGFR_ub_kcat)
Rule('catalyze_EGFR_dub', EGFR(ub='true') >> EGFR(ub='false'), catalyze_EGFR_dub_kcat);

## EGFR degradation and recycling
EGFR phosphorylation is not oly important for signal transduction, but also regulates the balance between slower, clathrin-independent and faster, clathrin-dependent receptor endocytosis <cite data-cite="2754712/IMG4CADM"></cite> <cite data-cite="2754712/ZES6PZM8"></cite> <cite data-cite="2754712/Q6YXK9FN"></cite>. 

We only implement clathrin-dependent endocytosis and assume that baseline clathrin-indepedent endocytosis and degradation is adequately captured by basal EGFR degradation rates. Endoyctosis is activated for phosphorylated EGFR, while receptor recycling is implemented independent of phosphorylation status. Endocytosis is implemented as compartment change from the plasma membrane `PM` to the endosome membrane `EM`. This implies that all binding partners that were located in the cytoplasm `CP` remain the the cytoplasm and thus continue signaling during endocytosis. Degradation of EGFR in endosomes is implemented to require ubiquitination. The degradation rate is implement as Hill-type kinetics with half maximal concentration `catalyze_EGFR_deg_kM` and propensity `catalyze_EGFR_endo_kcat`. Note that the default first order kinetics in BNG will multiply the rate by `ubEGFR`, so the overall rate increases with `ubEGFR`, but never exceeds `catalyze_EGFR_endo_kcat`. This ensures rate-limitation of clathrin dependent receptor degradation. The `delete_molecules=True` parameter encodes that only Egfr molecules are degradade and all other molecules are remain as possibly fragmented comlexes that can continue signaling.

In [11]:
Parameter('catalyze_EGFR_endo_kcat')
Parameter('catalyze_EGFR_deg_kM')
Parameter('catalyze_EGFR_recycl_kcat')
Parameter('catalyze_EGFR_deg_kcat')

Observable('ubEGFR', EGFR(Tyr='p') ** PM)

Expression('catalyze_EGFR_kendo', catalyze_EGFR_endo_kcat/(ubEGFR + catalyze_EGFR_deg_kM))

Rule('pEGFR_is_endocytosed', 
     EGFR(Tyr='p') ** PM >> EGFR(Tyr='p') ** EM, 
     catalyze_EGFR_kendo, move_connected=True)
Rule('EGFR_is_recycled', 
     EGFR() ** EM >> EGFR() ** PM, 
     catalyze_EGFR_recycl_kcat, move_connected=True)
Rule('ubEGFR_is_degraded', EGFR(ub='true') ** EM >> None, 
     catalyze_EGFR_deg_kcat, delete_molecules=True);

## SOS1 activation
Grb2 recruitement by Egfr also induces association of Sos1 to the plasma-membrane, where it can activate Ras by inducing nucleotide exchange <cite data-cite="2754712/AQG5TFL3"></cite>. In the model we implement that Grb2 and Sos1 preform complexes in the cytoplasm <cite data-cite="2754712/ITU427M5"></cite>. This interaction is also mediated by the GRB2 SH3 domain, which implies competitive binding between SOS1 and CBL. Sos1 is activated through association to the plasma-membrane, which is mediated recruitement of Grb2 by Egfr. Given that the training data does not include any data or perturbations at the Ras level, we implement a simple catalysis rule for activation of Ras, where SOS1 only binds GDP bound Ras and catalysis is implemented without homonucleotide exchange, allosteric feedback mechanisms or interactions with cdc25 <cite data-cite="2754712/AQG5TFL3"></cite>. Similarly, we implement inactivation of Ras through GTPase activation proteins, such as NF1, as a simple linear reaction with constant rate.

As previous binding reactions, the rules are paramterized using `kf` and `kD` values. As for the Grb2-Egfr interaction, the binding domains on Sos1 and Grb2 are implemented as `SH3` sites on both molecules, simplifying the much more intricate binding dynamics <cite data-cite="2754712/RB2FVNX9"></cite>. The translocation of Grb2 and Sos1 to the plasma membrane is not explicitely implemented in the model but implicitely encoded by requiring Grb2 and Egfr association for binding to Ras. To reduce correlations between parameter estimates, we implement the inactivation rate `[...]_gdp_kcat` of Ras as product of the activation rate `[...]_gtp_kcat` and a scaling factor `[...]_gdp_kcatr`. The requirement of `raf=None` in the inactivation rule will be explained in the Section on [Raf activation](#Raf-activation).

In [12]:
# binding Grb2-Sos1
Parameter('bind_GRB2_SOS1_kf', 10.0)
Parameter('bind_GRB2_SOS1_kD', 0.01)
Parameter('bind_GRB2_SOS1_phi', 1.0)

Expression('Gf_bind_GRB2_SOS1', 
           log(bind_GRB2_SOS1_kD))
Expression('Ea0_bind_GRB2_SOS1', 
           -bind_GRB2_SOS1_phi*log(bind_GRB2_SOS1_kD) - log(bind_GRB2_SOS1_kf))

Rule('GRB2_and_SOS1_bind_and_dissociate', 
     GRB2(SH3=None) + SOS1(SH3m=None) | GRB2(SH3=1) % SOS1(SH3m=1), 
     bind_GRB2_SOS1_phi, Ea0_bind_GRB2_SOS1, energy=True)
EnergyPattern('ep_bind_GRB2_SOS1', GRB2(SH3=1) % SOS1(SH3m=1), Gf_bind_GRB2_SOS1)
Annotation(GRB2_and_SOS1_bind_and_dissociate, 'http://identifiers.org/pubmed/7893993', 'isDescribedBy')

# binding Sos1-Ras
Parameter('bind_SOS1_RAS_kf')
Parameter('bind_SOS1_RAS_kD')
Expression('bind_SOS1_RAS_kr', bind_SOS1_RAS_kf*bind_SOS1_RAS_kD)
Rule('RTK_and_GRB2_bound_SOS1_binds_RASgdp', 
     GRB2(SH2=ANY, SH3=1) % SOS1(SH3m=1, ras=None) + RAS(sos1=None, state='gdp') 
     >> 
     GRB2(SH2=ANY, SH3=1) % SOS1(SH3m=1, ras=2) % RAS(sos1=2, state='gdp'), 
     bind_SOS1_RAS_kf)
Rule('SOS1_dissociates_from_RAS', 
     SOS1(ras=1) % RAS(sos1=1) >> SOS1(ras=None) + RAS(sos1=None), 
     bind_SOS1_RAS_kr)

# activation + inactivation Ras
Parameter('catalyze_SOS1_RAS_gtp_kcat')
Parameter('catalyze_NF1_RAS_gdp_kcatr')
Expression('catalyze_NF1_RAS_gdp_kcat', catalyze_SOS1_RAS_gtp_kcat*catalyze_NF1_RAS_gdp_kcatr)
Rule('SOS1_catalyzes_RAS_guanosine_exchange', 
     SOS1(ras=1) % RAS(sos1=1, state='gdp') >> SOS1(ras=None) + RAS(sos1=None, state='gtp'), 
     catalyze_SOS1_RAS_gtp_kcat)
Rule('RAS_hydrolysis_GTP', 
     RAS(raf=None, state='gtp') >> RAS(raf=None, state='gdp'), catalyze_NF1_RAS_gdp_kcat);

# ERK signaling
The Erk submodule describes the interaction between the two Rafs c-Raf and B-Raf, Mek (which describes Mek1 and Mek2) and Erk (which describes Erk1 and Erk2). Again, we instantiate the respective monomers and provide uniprot IDs as monomer annotations.

In [13]:
Monomer('BRAF', ['AA600', 'RBD', 'mek', 'raf', 'rafi'], {'AA600': ['E']})
Annotation(BRAF, 'http://identifiers.org/uniprot/P15056', 'is')

Monomer('CRAF', ['RBD', 'mek', 'raf', 'rafi'])
Annotation(CRAF, 'http://identifiers.org/uniprot/P04049', 'is')

Monomer('MEK', ['Dsite', 'meki', 'phospho', 'raf'], {'phospho': ['p', 'u']})
Annotation(MEK, 'http://identifiers.org/uniprot/Q02750', 'is')
Annotation(MEK, 'http://identifiers.org/uniprot/P36507', 'is')

Monomer('ERK', ['CD', 'phospho'], {'phospho': ['p', 'u']})
Annotation(ERK, 'http://identifiers.org/uniprot/P27361', 'is')
Annotation(ERK, 'http://identifiers.org/uniprot/P28482', 'is');

## Raf activation
GTP-bound Ras activates Raf by recruiting it to the membrane where it is phosphorylated and dimerizes. As understanding of the exact mechanisms of this activation process are still incomplete, we implemented a simplistic model of this process that captures essential features of the interaction. We implement a base dimerization rate between Raf monomers and only allow binding of Raf monomers to GTP-bound Ras. To implement the Ras-mediated dimerization of Raf, we implement an energy pattern for Ras-Raf tetramers modulates both binding affinities.

We implement both binding rules as energy rules, where we assume that affinities for B-Raf and C-Raf are the same for all interactions. As energy rules require symmetric forward and backward reactions, inactivation of Ras would protect Raf from unbinding from Ras. Accordingly, we add two additional inactivation reactions that dissociate Raf from Ras and required that no Raf was bound in the baseline inactivation.

In [14]:
# Raf-Raf binding
Parameter('bind_RAF_RAF_kf')
Parameter('bind_RAF_RAF_kD')
Parameter('bind_RAF_RAF_phi')
Expression('bind_BRAF_BRAF_kr', bind_RAF_RAF_kf*bind_RAF_RAF_kD)
Expression('Gf_bind_BRAF_BRAF', log(bind_RAF_RAF_kD))
Expression('Ea0_bind_BRAF_BRAF', -(bind_RAF_RAF_phi*log(bind_RAF_RAF_kD) + log(bind_RAF_RAF_kf)))
Expression('bind_BRAF_CRAF_kr', bind_RAF_RAF_kf*bind_RAF_RAF_kD)
Expression('Gf_bind_BRAF_CRAF', log(bind_RAF_RAF_kD))
Expression('Ea0_bind_BRAF_CRAF', -(bind_RAF_RAF_phi*log(bind_RAF_RAF_kD) + log(bind_RAF_RAF_kf)))
Expression('bind_CRAF_CRAF_kr', bind_RAF_RAF_kf*bind_RAF_RAF_kD)
Expression('Gf_bind_CRAF_CRAF', log(bind_RAF_RAF_kD))
Expression('Ea0_bind_CRAF_CRAF', -(bind_RAF_RAF_phi*log(bind_RAF_RAF_kD) + log(bind_RAF_RAF_kf)))
Rule('BRAF_and_BRAF_bind_and_dissociate', 
     BRAF(raf=None) + BRAF(raf=None) | BRAF(raf=1) % BRAF(raf=1), 
     bind_RAF_RAF_phi, 
     Ea0_bind_BRAF_BRAF, 
     energy=True)
Rule('BRAF_and_CRAF_bind_and_dissociate', 
     BRAF(raf=None) + CRAF(raf=None) | BRAF(raf=1) % CRAF(raf=1), 
     bind_RAF_RAF_phi, 
     Ea0_bind_BRAF_CRAF, 
     energy=True)
Rule('CRAF_and_CRAF_bind_and_dissociate', 
     CRAF(raf=None) + CRAF(raf=None) | CRAF(raf=1) % CRAF(raf=1), 
     bind_RAF_RAF_phi, 
     Ea0_bind_CRAF_CRAF, 
     energy=True)
EnergyPattern('ep_bind_BRAF_BRAF', BRAF(raf=1) % BRAF(raf=1), Gf_bind_BRAF_BRAF)
EnergyPattern('ep_bind_BRAF_CRAF', BRAF(raf=1) % CRAF(raf=1), Gf_bind_BRAF_CRAF)
EnergyPattern('ep_bind_CRAF_CRAF', CRAF(raf=1) % CRAF(raf=1), Gf_bind_CRAF_CRAF)

# Ras_gtp-RAF binding
Parameter('bind_RASstategtp_RAF_kf')
Parameter('bind_RASstategtp_RAF_kD')
Parameter('bind_RASstategtp_RAF_phi')
Expression('bind_RASstategtp_BRAF_kr', bind_RASstategtp_RAF_kf*bind_RASstategtp_RAF_kD)
Expression('Gf_bind_RASstategtp_BRAF', log(bind_RASstategtp_RAF_kD))
Expression('Ea0_bind_RASstategtp_BRAF', 
           -(bind_RASstategtp_RAF_phi*log(bind_RASstategtp_RAF_kD) + log(bind_RASstategtp_RAF_kf)))
Expression('bind_RASstategtp_CRAF_kr', bind_RASstategtp_RAF_kf*bind_RASstategtp_RAF_kD)
Expression('Gf_bind_RASstategtp_CRAF', log(bind_RASstategtp_RAF_kD))
Expression('Ea0_bind_RASstategtp_CRAF', 
           -(bind_RASstategtp_RAF_phi*log(bind_RASstategtp_RAF_kD) + log(bind_RASstategtp_RAF_kf)))
Rule('RASgtp_and_BRAF_bind_and_dissociate', 
     RAS(raf=None, state='gtp') + BRAF(RBD=None) | RAS(raf=1, state='gtp') % BRAF(RBD=1), 
     bind_RASstategtp_RAF_phi, 
     Ea0_bind_RASstategtp_BRAF, 
     energy=True)
Rule('RASgtp_and_CRAF_bind_and_dissociate', 
     RAS(raf=None, state='gtp') + CRAF(RBD=None) | RAS(raf=1, state='gtp') % CRAF(RBD=1), 
     bind_RASstategtp_RAF_phi, 
     Ea0_bind_RASstategtp_CRAF, 
     energy=True)
EnergyPattern('ep_bind_RASstategtp_BRAF', RAS(raf=1, state='gtp') % BRAF(RBD=1), Gf_bind_RASstategtp_BRAF)
EnergyPattern('ep_bind_RASstategtp_CRAF', RAS(raf=1, state='gtp') % CRAF(RBD=1), Gf_bind_RASstategtp_CRAF)

# Ras-Raf inactivation
Rule('GTP_hydrolysis_dissociates_BRAF_from_RAS', 
     RAS(raf=1, state='gtp') % BRAF(RBD=1) >> RAS(raf=None, state='gdp') + BRAF(RBD=None), 
     catalyze_NF1_RAS_gdp_kcat)
Rule('GTP_hydrolysis_dissociates_CRAF_from_RAS', 
     RAS(raf=1, state='gtp') % CRAF(RBD=1) >> RAS(raf=None, state='gdp') + CRAF(RBD=None),
     catalyze_NF1_RAS_gdp_kcat)

# Ras induces Raf dimerization
Parameter('ep_RAF_RAF_mod_RASstategtp_double_deltaG')
Expression('ep_RAF_RAF_mod_RASstategtp_double_Gf', log(ep_RAF_RAF_mod_RASstategtp_double_deltaG))
EnergyPattern('ep_BRAF_BRAF_mod_RAS_double', 
              RAS(raf=2, state='gtp') % BRAF(RBD=2, raf=1) % BRAF(RBD=3, raf=1) % RAS(raf=3, state='gtp'), 
              ep_RAF_RAF_mod_RASstategtp_double_Gf)
EnergyPattern('ep_BRAF_CRAF_mod_RAS_double', 
              RAS(raf=2, state='gtp') % BRAF(RBD=2, raf=1) % CRAF(RBD=3, raf=1) % RAS(raf=3, state='gtp'), 
              ep_RAF_RAF_mod_RASstategtp_double_Gf)
EnergyPattern('ep_CRAF_CRAF_mod_RAS_double', 
              RAS(raf=2, state='gtp') % CRAF(RBD=2, raf=1) % CRAF(RBD=3, raf=1) % RAS(raf=3, state='gtp'), 
              ep_RAF_RAF_mod_RASstategtp_double_Gf);

## Raf inhibition
Although Vemurafenib is an ATP-competitive inihibitor, it is known to exert allosteric effects. Here, we adapt a previously published model for these allosteric interactions between C-Raf, B-Raf and Vemurafenib <cite data-cite="2754712/7TJH2IAK"></cite>. The model implements binding between Vemurafenib (`RAFi`) and C-Raf and B-Raf (`RAF`) as energy rule as well as two sets of energy patterns that modulate `RAF-RAF` and `RAF-RAFi` affinities in single and double drug bound RAF-RAF homo and heterodimers.

In this implementation, C-Raf and B-Raf both correspond to kinase monomers `R` of the Kholodenko Core Model (KCM). `K1` of the KCM is equivalent to the parameter `bind_RAF_RAF_kD`, which was introduced in the previous Section. `K2` in the KCM is equivalent to `bind_RAFi_RAF_kD`, `f` to `ep_RAF_RAF_mod_RAFi_single_deltaG` and `g` to `ep_RAF_RAF_mod_RAFi_double_deltaG`.

In [15]:
# Binding RAFi-Raf
Parameter('bind_RAFi_RAF_kf')
Parameter('bind_RAFi_RAF_kD')
Parameter('bind_RAFi_RAF_phi')
Expression('bind_BRAFi_RAF_kr', bind_RAFi_RAF_kf*bind_RAFi_RAF_kD)
Expression('Gf_bind_BRAFi_RAF', log(bind_RAFi_RAF_kD))
Expression('Ea0_bind_BRAFi_RAF', -(bind_RAFi_RAF_phi*log(bind_RAFi_RAF_kD) + log(bind_RAFi_RAF_kf)))
Expression('bind_CRAFi_RAF_kr', bind_RAFi_RAF_kf*bind_RAFi_RAF_kD)
Expression('Gf_bind_CRAFi_RAF', log(bind_RAFi_RAF_kD))
Expression('Ea0_bind_CRAFi_RAF', -(bind_RAFi_RAF_phi*log(bind_RAFi_RAF_kD) + log(bind_RAFi_RAF_kf)))
Rule('RAFi_and_BRAF_bind_and_dissociate', 
     RAFi(raf=None) + BRAF(rafi=None) | RAFi(raf=1) % BRAF(rafi=1), 
     bind_RAFi_RAF_phi, 
     Ea0_bind_BRAFi_RAF, 
     energy=True)
Rule('RAFi_and_CRAF_bind_and_dissociate', 
     RAFi(raf=None) + CRAF(rafi=None) | RAFi(raf=1) % CRAF(rafi=1), 
     bind_RAFi_RAF_phi, 
     Ea0_bind_CRAFi_RAF, 
     energy=True)
EnergyPattern('ep_bind_BRAFi_RAF', RAFi(raf=1) % BRAF(rafi=1), Gf_bind_BRAFi_RAF)
EnergyPattern('ep_bind_CRAFi_RAF', RAFi(raf=1) % CRAF(rafi=1), Gf_bind_CRAFi_RAF)

# RAFi mediated affinity modulation
Parameter('ep_RAF_RAF_mod_RAFi_single_deltaG')
Parameter('ep_RAF_RAF_mod_RAFi_double_deltaG')
Expression('ep_RAF_RAF_mod_RAFi_single_Gf', log(ep_RAF_RAF_mod_RAFi_single_deltaG))
Expression('ep_RAF_RAF_mod_RAFi_double_Gf', 
           log(ep_RAF_RAF_mod_RAFi_double_deltaG) + log(ep_RAF_RAF_mod_RAFi_single_deltaG))

## Single Vemurafenib bound Raf dimers
EnergyPattern('ep_BRAF_BRAF_mod_RAFi_single', 
              BRAF(raf=1, rafi=None) % BRAF(raf=1, rafi=2) % RAFi(raf=2), 
              ep_RAF_RAF_mod_RAFi_single_Gf)
EnergyPattern('ep_BRAF_CRAF_mod_RAFi_single', 
              BRAF(raf=1, rafi=None) % CRAF(raf=1, rafi=2) % RAFi(raf=2), 
              ep_RAF_RAF_mod_RAFi_single_Gf)
EnergyPattern('ep_CRAF_BRAF_mod_RAFi_single', 
              CRAF(raf=1, rafi=None) % BRAF(raf=1, rafi=2) % RAFi(raf=2), 
              ep_RAF_RAF_mod_RAFi_single_Gf)
EnergyPattern('ep_CRAF_CRAF_mod_RAFi_single', 
              CRAF(raf=1, rafi=None) % CRAF(raf=1, rafi=2) % RAFi(raf=2), 
              ep_RAF_RAF_mod_RAFi_single_Gf)

## Double Vemurafenib bound Raf dimers
EnergyPattern('ep_BRAF_BRAF_mod_RAFi_double', 
              RAFi(raf=2) % BRAF(raf=1, rafi=2) % BRAF(raf=1, rafi=3) % RAFi(raf=3), 
              ep_RAF_RAF_mod_RAFi_double_Gf)
EnergyPattern('ep_BRAF_CRAF_mod_RAFi_double', 
              RAFi(raf=2) % BRAF(raf=1, rafi=2) % CRAF(raf=1, rafi=3) % RAFi(raf=3), 
              ep_RAF_RAF_mod_RAFi_double_Gf)
EnergyPattern('ep_CRAF_CRAF_mod_RAFi_double', 
              RAFi(raf=2) % CRAF(raf=1, rafi=2) % CRAF(raf=1, rafi=3) % RAFi(raf=3), 
              ep_RAF_RAF_mod_RAFi_double_Gf);

## Mek activation

Raf activation of Mek is mediated through phosphorylation at to S218 and S222 <cite data-cite="2754712/6B93WFLU"></cite>. As neither mass-spectrometry nor immunofluorescence measurements resolved individual phosphorylations on these sites, we decided to combine both into a single `phospho` site in the model. We implement the respective Mek phosphorylation as two step catalytic process including binding and phosphorylation. In agreement with previous studies, we assume that binding of Raf to Mek does not require prior activation of Raf, but is specific for unphosphorylated Mek <cite data-cite="2754712/KT8H2Y5J"></cite>. 

In [16]:
# Raf-Mek binding
Parameter('bind_RAF_MEKphosphou_kf')
Parameter('bind_RAF_MEKphosphou_kD')
Parameter('bind_RAF_MEKphosphou_phi')

Expression('bind_BRAF_MEKphosphou_kr', bind_RAF_MEKphosphou_kD*bind_RAF_MEKphosphou_kf)
Expression('Gf_bind_BRAF_MEKphosphou', log(bind_RAF_MEKphosphou_kD))
Expression('Ea0_bind_BRAF_MEKphosphou', 
           -bind_RAF_MEKphosphou_phi*log(bind_RAF_MEKphosphou_kD) - log(bind_RAF_MEKphosphou_kf))
Expression('bind_CRAF_MEKphosphou_kr', bind_RAF_MEKphosphou_kD*bind_RAF_MEKphosphou_kf)
Expression('Gf_bind_CRAF_MEKphosphou', log(bind_RAF_MEKphosphou_kD))
Expression('Ea0_bind_CRAF_MEKphosphou', 
           -bind_RAF_MEKphosphou_phi*log(bind_RAF_MEKphosphou_kD) - log(bind_RAF_MEKphosphou_kf))

Rule('BRAF_and_uMEK_bind_and_dissociate', 
     BRAF(mek=None) + MEK(phospho='u', raf=None) | BRAF(mek=1) % MEK(phospho='u', raf=1), 
     bind_RAF_MEKphosphou_phi, Ea0_bind_BRAF_MEKphosphou, energy=True)
Rule('CRAF_and_uMEK_bind_and_dissociate', 
     CRAF(mek=None) + MEK(phospho='u', raf=None) | CRAF(mek=1) % MEK(phospho='u', raf=1), 
     bind_RAF_MEKphosphou_phi, Ea0_bind_CRAF_MEKphosphou, energy=True)

EnergyPattern('ep_bind_BRAF_MEKphosphou', BRAF(mek=1) % MEK(phospho='u', raf=1), Gf_bind_BRAF_MEKphosphou)
EnergyPattern('ep_bind_CRAF_MEKphosphou', CRAF(mek=1) % MEK(phospho='u', raf=1), Gf_bind_CRAF_MEKphosphou);

For the activation step, we implement a physiological activation through Raf-Ras tetramers, which is implemented in four rules that account for all possible Raf homo- and heterodimer configurations, as well as oncogenic activation through B-Raf V600E mutation, which is implemented in four rules that account for B-Raf V600E molecules complexes that do not signal as dimers. In both cases, we require that the activating Raf molecule is not bound by an inhibitor and assume that kinetic rates are the same in both cases. 

Experimental data presented in this study indicated phospho-Mek accumulation for physiological signaling but not for oncogenic signaling, which is consistent with previous studies <cite data-cite="2754712/USPWXJI3"></cite> <cite data-cite="2754712/VFPC4KPB"></cite>, we implemented different phosphorylation rates depending on wheterh Mek is bound by an inhibitor. The exact molecular mechanisms for this difference between dimer mediated and V600E mediated are unknown, yet multiple studies point towards KSR, a scaffolding protein which was not included in the model, as a possible mediator of this difference <cite data-cite="2754712/9HXUM55P"></cite> <cite data-cite="2754712/YY7IQERJ"></cite>.

In [17]:
# phosphorylation
Parameter('catalyze_RAF_RAFrafiNone_MEK_p_kcat')
Parameter('catalyze_RAFrafiNone_MEKmekiNone_p_kcat')
Parameter('catalyze_RAFrafiNone_MEKmeki_MEKi_p_kcatr')

Expression('catalyze_RAFrafiNone_MEKmekiANY_p_kcat', 
           catalyze_RAFrafiNone_MEKmekiNone_p_kcat*catalyze_RAFrafiNone_MEKmeki_MEKi_p_kcatr)

## Raf-dimer mediated
Rule('BRAF_BRAF_phosphorylates_MEK', 
     MEK(phospho='u', raf=1) % BRAF(RBD=ANY, mek=1, raf=2, rafi=None) % BRAF(RBD=ANY, raf=2) 
     >> 
     MEK(phospho='p', raf=None) + BRAF(RBD=ANY, mek=None, raf=2, rafi=None) % BRAF(RBD=ANY, raf=2), 
     catalyze_RAF_RAFrafiNone_MEK_p_kcat)
Rule('BRAF_CRAF_phosphorylates_MEK', 
     MEK(phospho='u', raf=1) % BRAF(RBD=ANY, mek=1, raf=2, rafi=None) % CRAF(RBD=ANY, raf=2) 
     >> 
     MEK(phospho='p', raf=None) + BRAF(RBD=ANY, mek=None, raf=2, rafi=None) % CRAF(RBD=ANY, raf=2), 
     catalyze_RAF_RAFrafiNone_MEK_p_kcat)
Rule('CRAF_BRAF_phosphorylates_MEK', 
     MEK(phospho='u', raf=1) % CRAF(RBD=ANY, mek=1, raf=2, rafi=None) % BRAF(RBD=ANY, raf=2) 
     >> 
     MEK(phospho='p', raf=None) + CRAF(RBD=ANY, mek=None, raf=2, rafi=None) % BRAF(RBD=ANY, raf=2), 
     catalyze_RAF_RAFrafiNone_MEK_p_kcat)
Rule('CRAF_CRAF_phosphorylates_MEK', 
     MEK(phospho='u', raf=1) % CRAF(RBD=ANY, mek=1, raf=2, rafi=None) % CRAF(RBD=ANY, raf=2) 
     >> 
     MEK(phospho='p', raf=None) + CRAF(RBD=ANY, mek=None, raf=2, rafi=None) % CRAF(RBD=ANY, raf=2), 
     catalyze_RAF_RAFrafiNone_MEK_p_kcat)
## B-Raf V600E MEKi bound
Rule('BRAFV600E_phosphorylates_MEK_bound1', 
     MEK(meki=ANY, phospho='u', raf=1) % BRAF(AA600='E', mek=1, raf=None, rafi=None) 
     >> 
     MEK(meki=ANY, phospho='p', raf=None) + BRAF(AA600='E', mek=None, raf=None, rafi=None), 
     catalyze_RAFrafiNone_MEKmekiANY_p_kcat)
Rule('BRAFV600E_phosphorylates_MEK_bound2', 
     MEK(meki=ANY, phospho='u', raf=1) % BRAF(AA600='E', RBD=None, mek=1, raf=ANY, rafi=None) 
     >> 
     MEK(meki=ANY, phospho='p', raf=None) + BRAF(AA600='E', RBD=None, mek=None, raf=ANY, rafi=None), 
     catalyze_RAFrafiNone_MEKmekiANY_p_kcat)
Rule('BRAFV600E_phosphorylates_MEK_bound3', 
     MEK(meki=ANY, phospho='u', raf=1) % BRAF(AA600='E', RBD=ANY, mek=1, raf=2, rafi=None) % BRAF(RBD=None, raf=2)
     >>
     MEK(meki=ANY, phospho='p', raf=None) + BRAF(AA600='E', RBD=ANY, mek=None, raf=2, rafi=None) % BRAF(RBD=None, raf=2),
     catalyze_RAFrafiNone_MEKmekiANY_p_kcat)
Rule('BRAFV600E_phosphorylates_MEK_bound4',
     MEK(meki=ANY, phospho='u', raf=1) % BRAF(AA600='E', RBD=ANY, mek=1, raf=2, rafi=None) % CRAF(RBD=None, raf=2)
     >>
     MEK(meki=ANY, phospho='p', raf=None) + BRAF(AA600='E', RBD=ANY, mek=None, raf=2, rafi=None) % CRAF(RBD=None, raf=2),
     catalyze_RAFrafiNone_MEKmekiANY_p_kcat)
## B-Raf V600E MEKi unbound
Rule('BRAFV600E_phosphorylates_MEK_unbound1', 
     MEK(meki=None, phospho='u', raf=1) % BRAF(AA600='E', mek=1, raf=None, rafi=None)
     >>
     MEK(meki=None, phospho='p', raf=None) + BRAF(AA600='E', mek=None, raf=None, rafi=None),
     catalyze_RAFrafiNone_MEKmekiNone_p_kcat)
Rule('BRAFV600E_phosphorylates_MEK_unbound2',
     MEK(meki=None, phospho='u', raf=1) % BRAF(AA600='E', RBD=None, mek=1, raf=ANY, rafi=None)
     >>
     MEK(meki=None, phospho='p', raf=None) + BRAF(AA600='E', RBD=None, mek=None, raf=ANY, rafi=None),
     catalyze_RAFrafiNone_MEKmekiNone_p_kcat)
Rule('BRAFV600E_phosphorylates_MEK_unbound3',
     MEK(meki=None, phospho='u', raf=1) % BRAF(AA600='E', RBD=ANY, mek=1, raf=2, rafi=None) % BRAF(RBD=None, raf=2)
     >>
     MEK(meki=None, phospho='p', raf=None) + BRAF(AA600='E', RBD=ANY, mek=None, raf=2, rafi=None) % BRAF(RBD=None,raf=2),
     catalyze_RAFrafiNone_MEKmekiNone_p_kcat)
Rule('BRAFV600E_phosphorylates_MEK_unbound4',
     MEK(meki=None, phospho='u', raf=1) % BRAF(AA600='E', RBD=ANY, mek=1, raf=2, rafi=None) % CRAF(RBD=None, raf=2)
     >>
     MEK(meki=None, phospho='p', raf=None) + BRAF(AA600='E', RBD=ANY, mek=None, raf=2, rafi=None) % CRAF(RBD=None, raf=2),
     catalyze_RAFrafiNone_MEKmekiNone_p_kcat);

PP2A and PPP1 have been suggested as putative phosphatases of Mek1, but exact regulatory mechanisms remain poorly understood. Accordingly we implement MEK dephosphorylation as conversion rule with constant rate.

In [18]:
# dephosphorylation
Parameter('catalyze_PP2A_MEK_u_kcatr')
Expression('catalyze_PP2A_MEK_u_kcat', catalyze_PP2A_MEK_u_kcatr*catalyze_RAF_RAFrafiNone_MEK_p_kcat)
Rule('MEK_is_dephosphorylated', MEK(phospho='p') >> MEK(phospho='u'), catalyze_PP2A_MEK_u_kcat);

## Mek inhibition
Cobimetinib is an allosteric (type III) inhibitor that potently and selectively inhibits Erk activation when binding Mek. Recent studies reported a distinct potency of allosteric Mek inhibitor for Ras-GTP versus B-Raf V600E driven Mek signaling. One component of this difference in potency is the accumulation of phosphorylated Mek in Ras-GTP driven signaling, but not in B-Raf V600E driven signaling that was already described in the previous Section. A second component is the differential affinity of allosteric Mek inihibitors towards phosphorylated MEK <cite data-cite="2754712/FQ6JM9RD"></cite> <cite data-cite="undefined"></cite>. Accordingly, we implement binding as energy rule and add an energy pattern that allows Mek phosphorylation to modulation the respective affinity.

In [19]:
# MEKi binding
Parameter('bind_MEKi_MEK_kf')
Parameter('bind_MEKi_MEK_kD')
Parameter('bind_MEKi_MEK_phi')
Expression('bind_MEKi_MEK_kr', bind_MEKi_MEK_kf*bind_MEKi_MEK_kD)
Expression('Gf_bind_MEKi_MEK', log(bind_MEKi_MEK_kD))
Expression('Ea0_bind_MEKi_MEK', -(bind_MEKi_MEK_phi*log(bind_MEKi_MEK_kD) + log(bind_MEKi_MEK_kf)))
Rule('MEKi_and_MEK_bind_and_dissociate', 
     MEKi(mek=None) + MEK(meki=None) | MEKi(mek=1) % MEK(meki=1), 
     bind_MEKi_MEK_phi, 
     Ea0_bind_MEKi_MEK, 
     energy=True)
EnergyPattern('ep_bind_MEKi_MEK', MEKi(mek=1) % MEK(meki=1), Gf_bind_MEKi_MEK);

# Modulated pMEK affinity 
Parameter('ep_MEKphosphop_MEKi_deltaG')
Expression('ep_MEKphosphop_MEKi_Gf', log(ep_MEKphosphop_MEKi_deltaG))
EnergyPattern('ep_MEKphosphop_MEKi_single', MEK(meki=1, phospho='p') % MEKi(mek=1), ep_MEKphosphop_MEKi_Gf);

## Erk activation
Mek phosphorylates Erk1 at T202 and T185 and Erk2 at Y204 and Y187. As for Mek phosphorylation sites, neither mass-spectrometry nor immunofluorescence measurements resolved individual phosphorylations, both sites were combined into a single `phospho` site in the model. 

As for the Mek phosphorylation we implement Erk phosphorylation as two step catalysis reaction. The binding between Mek and Erk is mediated through interaction of the MAPK-Docking site on the N-Terminus of MEK (`Dsite` site in the model) <cite data-cite="2754712/5D9BPF8E"></cite> <cite data-cite="2754712/L5AI98Y6"></cite> with the common docking domain on Erk (`CD` site in the model) <cite data-cite="2754712/VILTI493"></cite> <cite data-cite="2754712/PN4VMMJP"></cite>. In combination with Mek's nuclear export sequence, this interaction allows Mek to act as a cytosolic anchor for inactive Erk <cite data-cite="2754712/L5AI98Y6"></cite>, which suggests that this interaction is not dependent of Mek phosphorylation. After Mek activation, Erk releases from Mek and shuttles in and out of the nucleus, which suggests that affinity between Mek and Erk depends on Erk phosphorylation <cite data-cite="2754712/JAB2JXXQ"></cite> <cite data-cite="2754712/SI73C6VD"></cite> <cite data-cite="2754712/WPX829YQ"></cite>. In the model, we implement the dependence on Erk phosphorylation as binary interaction, where Mek only binds unphosphorylated Erk. Nuclear shuttling of Mek and Erk was not implemented in the model.

In [20]:
# Mek-Erk binding
Parameter('bind_MEK_ERKphosphou_kf')
Parameter('bind_MEK_ERKphosphou_kD')
Expression('bind_MEK_ERKphosphou_kr', bind_MEK_ERKphosphou_kf*bind_MEK_ERKphosphou_kD)
Rule('MEK_binds_uERK', 
     MEK(Dsite=None) + ERK(CD=None, phospho='u') >> MEK(Dsite=1) % ERK(CD=1, phospho='u'), 
     bind_MEK_ERKphosphou_kf)
Rule('MEK_dissociates_from_ERK', 
     MEK(Dsite=1) % ERK(CD=1) >> MEK(Dsite=None) + ERK(CD=None), 
     bind_MEK_ERKphosphou_kr)

# Erk phosphorylation
Parameter('catalyze_MEKmekiNone_phosphop_ERK_p_kcat')
Rule('pMEK_phosphorylates_ERK', 
     ERK(CD=1, phospho='u') % MEK(Dsite=1, meki=None, phospho='p') 
     >>
     ERK(CD=None, phospho='p') + MEK(Dsite=None, meki=None, phospho='p'), 
     catalyze_MEKmekiNone_phosphop_ERK_p_kcat);

# Feedback mechanisms
Activation of Erk through phosphorylation induces a variety of different negative feedback mechanisms that inhibit signal transduction in Egfr and Erk pathways <cite data-cite="2754712/48YU6P7U"></cite> <cite data-cite="2754712/URVR5RD5"></cite>. In this module we specifically implement four of these mechanisms that, specifically the increase in Spry, Egfr and Dusp expression levels as well as phosphorylation of Sos1. For this purpose we introduce two new monomers, namely Dusp (which describes Dusp4 and Dusp6) and Spry (which describes Spry2 and Spry4).

Some of the negative feedback mechanisms that were described in previous studies are omitted in this model, including Erk negative feedback phosphorylations on Mek, and Raf. As the timescale of phosphorylation reactions is in the order of seconds, a high inhibitory potency of these feedback mechanisms should give rise to phospho-Erk pulses on a timescale of seconds. Yet the experimentally observed timescale of phospho-Erk pulses was in the order of minutes, which suggests that it is unlikely that phosphorylations feedbacks are the primary determinant of the pulse shape. This hypothesis is backed by previous studies on phospho-turnover in Egfr mediated signaling <cite data-cite="2754712/KWBTHZEK"></cite>. Accordingly we only expect a subtle effect of the negative feedback mechanisms, which would be difficult to be resolve in the model, given that no experimental data on respective phosphorylation sites or Ras-GTP levels were available.

In [21]:
Monomer('DUSP', ['erk'])
Annotation(DUSP, 'http://identifiers.org/uniprot/Q13115', 'is')
Annotation(DUSP, 'http://identifiers.org/uniprot/Q16828', 'is')

Monomer('mDUSP')
Annotation(mDUSP, 'http://identifiers.org/hgnc/3070', 'is')
Annotation(mDUSP, 'http://identifiers.org/hgnc/3072', 'is')

Monomer('mEGFR')
Annotation(mEGFR, 'http://identifiers.org/hgnc/3236', 'is')

Monomer('SPRY', ['SH3m'])

Annotation(SPRY, 'http://identifiers.org/uniprot/O43597', 'is')
Annotation(SPRY, 'http://identifiers.org/uniprot/Q9C004', 'is')

Monomer('mSPRY')
Annotation(mSPRY, 'http://identifiers.org/hgnc/11270', 'is')
Annotation(mSPRY, 'http://identifiers.org/hgnc/15533', 'is');

## Spry expression
The proteomic data collected in this study confirmed previous findings that phosphorylation of Erk enhances expression of Spry. Increase in Spry expression inhibits Egfr signaling and is mediated by regulation of downstream transcription factors <cite data-cite="2754712/URVR5RD5"></cite>. Specifically, Spry competes with Sos1 for the binding of Grb2 <cite data-cite="2754712/V8N4ML4G"></cite> <cite data-cite="2754712/HWMDC2F4"></cite>. Accordingly higher levels of Spry can antagonize Sos1 signaling by sequestering Grb2 in Spry:Grb2 complexes. 

Here, we implement basal synthesis and degradation reactions for Spry, which are parameterised with `[...]_eq` and `[...]_kdeg` parameters to improve parameter identifiability. Regulation of synthesis rate is implemented via multiplicative factors, which yields log-additive transcription levels. To avoid correlations of parameter estimates, we normalize this expression rate with the basal Spry degradation rate.

In [22]:
# basal expression and degradation
Parameter('mSPRY_eq')
Parameter('SPRY_eq')
Parameter('synthesize_ERKphosphop_SPRY_ERK_gexpslope')
Parameter('msynthesize_ERKphosphop_SPRY_kdeg')
Parameter('psynthesize_ERKphosphop_SPRY_kdeg')
Parameter('synthesize_ERKphosphop_SPRY_ERK_kM')

Expression('msynthesize_ERKphosphop_SPRY_kbase', 
           mSPRY_eq*psynthesize_ERKphosphop_SPRY_kdeg)
Expression('psynthesize_ERKphosphop_SPRY_kbase', 
           1000000.0*SPRY_eq*psynthesize_ERKphosphop_SPRY_kdeg/(N_Avogadro*volume*mSPRY_eq))
Expression('psynthesize_ERKphosphop_SPRY_ksyn', 
           psynthesize_ERKphosphop_SPRY_kbase)

Observable('modulation_synthesize_ERKphosphop_SPRY_ERK', ERK(phospho='p'))

Expression('synthesize_ERKphosphop_SPRY_ERK_kmod', 
           modulation_synthesize_ERKphosphop_SPRY_ERK*synthesize_ERKphosphop_SPRY_ERK_gexpslope/(modulation_synthesize_ERKphosphop_SPRY_ERK + synthesize_ERKphosphop_SPRY_ERK_kM) + 1)
Expression('msynthesize_ERKphosphop_SPRY_ksyn', 
           1.0*msynthesize_ERKphosphop_SPRY_kbase*synthesize_ERKphosphop_SPRY_ERK_kmod)

Rule('synthesis_mSPRY', 
     None >> mSPRY() ** CP, 
     msynthesize_ERKphosphop_SPRY_ksyn)
Rule('basal_degradation_mSPRY', mSPRY() >> None, 
     msynthesize_ERKphosphop_SPRY_kdeg, delete_molecules=True)
Rule('synthesis_pSPRY', mSPRY() ** CP >> mSPRY() ** CP + SPRY(SH3m=None) ** CP, 
     psynthesize_ERKphosphop_SPRY_ksyn)
Rule('basal_degradation_pSPRY', SPRY() >> None, 
     psynthesize_ERKphosphop_SPRY_kdeg, delete_molecules=True);

The binding between Spry and Grb2 is parametrized using `[...]_kD` and `[...]_kf` parameters as previously described. Competitive binding between Spry and Sos1 for Grb2 is implemented by specifying the same binding site on Grb2 (`SH3`) for the binding of both reactions.

In [23]:
# Spry-Grb2 binding
Parameter('bind_SPRY_GRB2_kf')
Parameter('bind_SPRY_GRB2_kD')
Parameter('bind_SPRY_GRB2_phi')

Expression('bind_SPRY_GRB2_kr', 
           bind_SPRY_GRB2_kD*bind_SPRY_GRB2_kf)
Expression('Gf_bind_SPRY_GRB2', 
           log(bind_SPRY_GRB2_kD))
Expression('Ea0_bind_SPRY_GRB2', 
           -bind_SPRY_GRB2_phi*log(bind_SPRY_GRB2_kD) - log(bind_SPRY_GRB2_kf))

Rule('SPRY_and_GRB2_bind_and_dissociate', 
     SPRY(SH3m=None) + GRB2(SH3=None) | SPRY(SH3m=1) % GRB2(SH3=1), 
     bind_SPRY_GRB2_phi, Ea0_bind_SPRY_GRB2, energy=True)

EnergyPattern('ep_bind_SPRY_GRB2', 
              SPRY(SH3m=1) % GRB2(SH3=1), Gf_bind_SPRY_GRB2);

## Sos1 phosphorylation
Phospho-proteomic data collected in this study confirmed previous findings that phosphorylation of Erk enhances phosphorylation of Sos1 at S1134. Phosphorylation of Sos1 inhibits Egfr signaling and is mediated through upregulation of Rsk expression and phosphorylation <cite data-cite="2754712/URVR5RD5"></cite>. Specifically, it creates a docking site for 14-3-3 on Sos1 and thereby sequestering it from Grb2 <cite data-cite="2754712/JPLX2K5C"></cite>.

We do not include Rsk or 14-3-3 in the model as absolute abundances were measured for neither of proteins. Instead we decided to implement basal phosphorylation and dephosphorylation rules as well as an phospho-Erk dependent phosphorylation rule for which the phosphorylation linearly depends on Erk phosphorylation. The baseline rules were implemented using constant rate `[...]_kbase` and `[...]_kcat` respectively. To avoid parameter correlations, the phospho-Erk dependent phosphorylation rate was normalized by the baseline phosphorylation rate.

In [24]:
# base phosphorylation rate
Parameter('catalyze_ERKphosphop_SOS1_pS1134_kbase')
Parameter('catalyze_ERKphosphop_SOS1_pS1134_kcat')
Parameter('catalyze_phosphatase_SOS1_uS1134_kcatr')
Expression('catalyze_phosphatase_SOS1_uS1134_kcat', 
           catalyze_ERKphosphop_SOS1_pS1134_kcat*catalyze_phosphatase_SOS1_uS1134_kcatr)
Rule('SOS1_is_phosphorylated', 
     SOS1(S1134='u') >> SOS1(S1134='p'), 
     catalyze_ERKphosphop_SOS1_pS1134_kbase)
Rule('SOS1_is_dephosphorylated', 
     SOS1(S1134='p') >> SOS1(S1134='u'), 
     catalyze_phosphatase_SOS1_uS1134_kcat)
Rule('pERK_phosphorylates_SOS1', 
     ERK(phospho='p') + SOS1(S1134='u') >> ERK(phospho='p') + SOS1(S1134='p'), 
     catalyze_ERKphosphop_SOS1_pS1134_kcat)

Rule('pERK_phosphorylates_SOS1', ERK(phospho='p') + SOS1(S1134='u') >> ERK(phospho='p') + SOS1(S1134='p'), catalyze_ERKphosphop_SOS1_pS1134_kcat)

The sequestration and resulting loss of affinity for GRB2 was implemented using an energy patterns. 

In [25]:
Parameter('ep_SOS1S1134p_GRB2_deltaG')
Expression('ep_SOS1S1134p_GRB2_Gf', log(ep_SOS1S1134p_GRB2_deltaG))
EnergyPattern('ep_SOS1S1134p_GRB2_single', 
              SOS1(S1134='p', SH3m=1) % GRB2(SH3=1), ep_SOS1S1134p_GRB2_Gf)

EnergyPattern('ep_SOS1S1134p_GRB2_single', SOS1(S1134='p', SH3m=1) % GRB2(SH3=1), ep_SOS1S1134p_GRB2_Gf)

## Egfr expression
Proteomic data collected in this study confirmed previous findings that phosphorylation of Erk increases Egfr expression. Increased Egfr expression promotes Egfr signaling and is mediated through regulation of downstream transcription factors <cite data-cite="2754712/URVR5RD5"></cite>. Regulation of EGFR synthesis is implemented in the same way as regulation of SPRY synthesis.

In [26]:
Parameter('mEGFR_eq')
Parameter('EGFR_eq')
Parameter('synthesize_ERKphosphop_EGFR_ERK_gexpslope')
Parameter('msynthesize_ERKphosphop_EGFR_kdeg')
Parameter('psynthesize_ERKphosphop_EGFR_kdeg')
Parameter('EGFR_crispr')
Parameter('synthesize_ERKphosphop_EGFR_ERK_kM')

Expression('msynthesize_ERKphosphop_EGFR_kbase', 
           mEGFR_eq*psynthesize_ERKphosphop_EGFR_kdeg)
Expression('psynthesize_ERKphosphop_EGFR_kbase', 
           1000000.0*EGFR_eq*psynthesize_ERKphosphop_EGFR_kdeg/(N_Avogadro*volume*mEGFR_eq))
Expression('psynthesize_ERKphosphop_EGFR_ksyn', 
           psynthesize_ERKphosphop_EGFR_kbase)


Observable('modulation_synthesize_ERKphosphop_EGFR_ERK', ERK(phospho='p'))

Expression('synthesize_ERKphosphop_EGFR_ERK_kmod', 
           modulation_synthesize_ERKphosphop_EGFR_ERK*synthesize_ERKphosphop_EGFR_ERK_gexpslope/(modulation_synthesize_ERKphosphop_EGFR_ERK + synthesize_ERKphosphop_EGFR_ERK_kM) + 1)
Expression('msynthesize_ERKphosphop_EGFR_ksyn', 
           msynthesize_ERKphosphop_EGFR_kbase*synthesize_ERKphosphop_EGFR_ERK_kmod*EGFR_crispr)


Rule('synthesis_mEGFR', 
     None >> mEGFR() ** PM, 
     msynthesize_ERKphosphop_EGFR_ksyn)
Rule('basal_degradation_mEGFR', 
     mEGFR() >> None, 
     msynthesize_ERKphosphop_EGFR_kdeg, delete_molecules=True)
Rule('synthesis_pEGFR', 
     mEGFR() ** PM >> mEGFR() ** PM + EGFR(KD=None, Tyr='u', rtkf=None, ub='false') ** PM, 
     psynthesize_ERKphosphop_EGFR_ksyn)
Rule('basal_degradation_pEGFR', 
     EGFR() >> None, 
     psynthesize_ERKphosphop_EGFR_kdeg, delete_molecules=True);

## Dusp expression
The proteomic data collected in this study confirmed previous findings that phosphorylation of Erk enhances expression of Dusp. Increase in Dusp expression inhibits Erk signaling and is mediated by regulation of downstream transcription factors <cite data-cite="2754712/URVR5RD5"></cite>. Specifically, Dusp are phosphatases that dephosphorylated Erk on the same residues that are phosphorylated by Mek. In the proteomic data we individually measured Dusp4 and Dusp6 abundances. Dusp4 has primarily nuclear localisation <cite data-cite="2754712/EZR5VHIS"></cite>, while Dusp6 has primarily cytosolic localisation <cite data-cite="2754712/UPE8FY25"></cite>. As we did not implement shuttling of Erk between the nucleus and cytosol, we lumped Dusp4 and Dusp6 species. Regulation of DUSP synthesis is implemented in the same way as regulation of SPRY synthesis

In [27]:
# expression
Parameter('mDUSP_eq')
Parameter('DUSP_eq')
Parameter('synthesize_ERKphosphop_DUSP_ERK_gexpslope')
Parameter('msynthesize_ERKphosphop_DUSP_kdeg')
Parameter('psynthesize_ERKphosphop_DUSP_kdeg')
Parameter('synthesize_ERKphosphop_DUSP_ERK_kM')

Expression('msynthesize_ERKphosphop_DUSP_kbase', 
           mDUSP_eq*psynthesize_ERKphosphop_DUSP_kdeg)
Expression('psynthesize_ERKphosphop_DUSP_kbase', 
           1000000.0*DUSP_eq*psynthesize_ERKphosphop_DUSP_kdeg/(N_Avogadro*volume*mDUSP_eq))
Expression('psynthesize_ERKphosphop_DUSP_ksyn', 
           psynthesize_ERKphosphop_DUSP_kbase)

Observable('modulation_synthesize_ERKphosphop_DUSP_ERK', 
           ERK(phospho='p'))

Expression('synthesize_ERKphosphop_DUSP_ERK_kmod', 
           modulation_synthesize_ERKphosphop_DUSP_ERK*synthesize_ERKphosphop_DUSP_ERK_gexpslope/(modulation_synthesize_ERKphosphop_DUSP_ERK + synthesize_ERKphosphop_DUSP_ERK_kM) + 1)
Expression('msynthesize_ERKphosphop_DUSP_ksyn', 
           1.0*msynthesize_ERKphosphop_DUSP_kbase*synthesize_ERKphosphop_DUSP_ERK_kmod)


Rule('synthesis_mDUSP', 
     None >> mDUSP() ** CP, 
     msynthesize_ERKphosphop_DUSP_ksyn)
Rule('basal_degradation_mDUSP', 
     mDUSP() >> None, 
     msynthesize_ERKphosphop_DUSP_kdeg, delete_molecules=True)
Rule('synthesis_pDUSP', 
     mDUSP() ** CP >> mDUSP() ** CP + DUSP(erk=None) ** CP, 
     psynthesize_ERKphosphop_DUSP_ksyn)
Rule('basal_degradation_pDUSP', 
     DUSP() >> None, 
     psynthesize_ERKphosphop_DUSP_kdeg, delete_molecules=True);

We implement the dephosphorylation as two-step catalytic process. As both Dusps interact with Erk via the common docking domain that also mediates interaction between Mek to Erk <cite data-cite="2754712/VILTI493"></cite>, the binding was implemented using the same site on the Erk (`CD`) monomer. This could lead to competitive binding between Mek and Erk, yet as Dusp, in contrast to Mek, seem to preferentially bind the phosphorylated form of Erk <cite data-cite="2754712/LWCCZVEV"></cite>, we constrained binding of Dusp to Erk on Erk phosphorylation, which makes the two binding reactions independent.

In [28]:
# Dusp-Erk binding
Parameter('bind_DUSP_ERKphosphop_kf')
Parameter('bind_DUSP_ERKphosphop_kD')
Expression('bind_DUSP_ERKphosphop_kr', bind_DUSP_ERKphosphop_kf*bind_DUSP_ERKphosphop_kD)
Rule('DUSP_binds_pERK', 
     DUSP(erk=None) + ERK(CD=None, phospho='p') >> DUSP(erk=1) % ERK(CD=1, phospho='p'), 
     bind_DUSP_ERKphosphop_kf)
Rule('DUSP_dissociates_from_ERK', 
     DUSP(erk=1) % ERK(CD=1) >> DUSP(erk=None) + ERK(CD=None), 
     bind_DUSP_ERKphosphop_kr)

# Erk dephosphorylation
Parameter('catalyze_DUSP_ERK_u_kcatr')
Expression('catalyze_DUSP_ERK_u_kcat', catalyze_MEKmekiNone_phosphop_ERK_p_kcat*catalyze_DUSP_ERK_u_kcatr)
Rule('DUSP_dephosphorylates_ERK', 
     ERK(CD=1, phospho='p') % DUSP(erk=1) >> ERK(CD=None, phospho='u') + DUSP(erk=None), 
     catalyze_DUSP_ERK_u_kcat);

# Initialization
For all monomers for which we did not implement expression and basal degradation rules we implemented initial abundances that were assumed to be constant in all experiments. To simplify specification of estimation boundaries for these parameters, these parameters were assumed to be specified in [molecules/cell] and then accordingly transformed to yield model initializations in [uM]. 

In [29]:
Parameter('BRAF_0')
Parameter('CRAF_0')
Parameter('RAS_0')
Parameter('MEK_0')
Parameter('ERK_0')
Parameter('GRB2_0')
Parameter('SOS1_0')
Parameter('CBL_0')

Expression('initBRAF', 1000000.0*BRAF_0*(N_Avogadro*volume)**(-1))
Expression('initCRAF', 1000000.0*CRAF_0*(N_Avogadro*volume)**(-1))
Expression('initRAS', 1000000.0*RAS_0*(N_Avogadro*volume)**(-1))
Expression('initMEK', 1000000.0*MEK_0*(N_Avogadro*volume)**(-1))
Expression('initERK', 1000000.0*ERK_0*(N_Avogadro*volume)**(-1))
Expression('initGRB2', 1000000.0*GRB2_0*(N_Avogadro*volume)**(-1))
Expression('initSOS1', 1000000.0*SOS1_0*(N_Avogadro*volume)**(-1))
Expression('initCBL', 1000000.0*CBL_0*(N_Avogadro*volume)**(-1))

Initial(BRAF(AA600='E', RBD=None, mek=None, raf=None, rafi=None) ** CP, initBRAF)
Initial(CRAF(RBD=None, mek=None, raf=None, rafi=None) ** CP, initCRAF)
Initial(RAS(raf=None, sos1=None, state='gdp') ** CP, initRAS)
Initial(MEK(Dsite=None, meki=None, phospho='u', raf=None) ** CP, initMEK)
Initial(ERK(CD=None, phospho='u') ** CP, initERK)
Initial(GRB2(SH2=None, SH3=None) ** CP, initGRB2)
Initial(SOS1(S1134='u', SH3m=None, ras=None) ** CP, initSOS1)
Initial(CBL(SH3m=None) ** CP, initCBL)

Initial(CBL(SH3m=None) ** CP, initCBL)

# Link to Experimental Data
To calibrate the model to experimental data we create a series of expression that account for normalizations in data generation. For absolute proteomic measurements we create expressions `t[...]_obs` that convert the unit of model simulations from [uM] to [molecules/cell]. For phospho-proteomic measurements we create expressions `p[...]_obs` that are equal to phospho-abundances normalized by total abundances. For transcriptomic measurements we create expressions `tm[...]_obs` that describe transcript abundances in [FPKM]. For both proteomic and transcriptomic measurements we log-transfrom observables to reflect respective transformations in the experimental data. Note that this effectively yields a log-normal error model. For immuno-fluorescence measurements we created expressions `p[...]_IF_obs` that were first normalized by total abundances and then scaled by a mulitplicative factor `[...]_IF_scale` and an additive factor `[...]_IF_offset`. For species that describe multiple protein isoforms, absolute proteomic data was averaged over both isoforms.

In [30]:
# proteomic measurements
Observable('tBRAF', BRAF())
Observable('tCRAF', CRAF())
Observable('tRAS', RAS())
Observable('tMEK', MEK())
Observable('tERK', ERK())
Observable('tDUSP', DUSP())
Observable('tEGFR', EGFR())
Observable('tGRB2', GRB2())
Observable('tSPRY', SPRY())
Observable('tSOS1', SOS1())
Observable('tCBL', CBL())
Expression('tBRAF_obs', log(1.0e-6*N_Avogadro*volume*tBRAF))
Expression('tCRAF_obs', log(1.0e-6*N_Avogadro*volume*tCRAF))
Expression('tRAS_obs', log(1.0e-6*N_Avogadro*volume*tRAS))
Expression('tMEK_obs', log(1.0e-6*N_Avogadro*volume*tMEK))
Expression('tERK_obs', log(1.0e-6*N_Avogadro*volume*tERK))
Expression('tDUSP_obs', log(1.0e-6*N_Avogadro*volume*tDUSP))
Expression('tEGFR_obs', log(1.0e-6*N_Avogadro*volume*tEGFR))
Expression('tGRB2_obs', log(1.0e-6*N_Avogadro*volume*tGRB2))
Expression('tSPRY_obs', log(1.0e-6*N_Avogadro*volume*tSPRY))
Expression('tSOS1_obs', log(1.0e-6*N_Avogadro*volume*tSOS1))
Expression('tCBL_obs', log(1.0e-6*N_Avogadro*volume*tCBL))

# phospho-proteomic measurements
Observable('pERK', ERK(phospho='p'))
Observable('pS1134SOS1', SOS1(S1134='p'))
Expression('pERK_obs', pERK/tERK)
Expression('pS1134SOS1_obs', pS1134SOS1/tSOS1)

# transcriptomic measurements
Observable('tmDUSP', mDUSP())
Observable('tmEGFR', mEGFR())
Observable('tmSPRY', mSPRY())
Expression('tmDUSP_obs', log(tmDUSP))
Expression('tmEGFR_obs', log(tmEGFR))
Expression('tmSPRY_obs', log(tmSPRY))

# immunofluorescence measurements
Parameter('pMEK_IF_scale')
Parameter('pMEK_IF_offset')
Parameter('pERK_IF_scale')
Parameter('pERK_IF_offset')
Observable('pMEK', MEK(phospho='p'))
Expression('pMEK_obs', pMEK/tMEK)
Expression('pERK_IF_obs', pERK_IF_offset + pERK_IF_scale*pERK_obs)
Expression('pMEK_IF_obs', pMEK_IF_offset + pMEK_IF_scale*pMEK_obs);

In [31]:
# validate the model constructed here is equivalent to the model used for the study
import sys
import os
import MARM
sys.path.insert(0, os.path.join(MARM.paths.get_directory(), 'pysb_flat'))
from RTKERK__base import model as ref_model

for comp in ref_model.components:
    assert comp.name in model.components.keys() or isinstance(comp, (Observable, Expression))
    if comp.name in model.components.keys():
        assert str(comp) == str(model.components[comp.name]) or isinstance(comp, Parameter)

for comp in model.components:
    assert comp.name in ref_model.components.keys()

# Bibliography
<div class="cite2c-biblio"></div>